This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo [parmest](https://pyomo.readthedocs.io/en/stable/contributed_packages/parmest/driver.html).

In [1]:
# Import libraries
from pyomo.environ import *
from pyomo.dae import *
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
from pyomo.contrib.interior_point.interface import InteriorPointInterface

This example has a series reaction $A \rightarrow B \rightarrow C$. The dataset consists of measures concentrations of A, B and C over time. The goal is to estimate the rate constants $k_1$ and $k_2$ for the two reactions.

In [2]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

,t,ca,cb,cc
0,0.000,0.957,-0.031,-0.015
1,0.263,0.557,0.330,0.044
2,0.526,0.342,0.512,0.156
3,0.789,0.224,0.499,0.310
4,1.053,0.123,0.428,0.454


In [3]:
# Convert data to a list of dictionaries
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]

In [4]:
#
# Define the model 
#
def ABC_model(data):
    
    ca_meas = data['ca_meas']
    cb_meas = data['cb_meas']
    cc_meas = data['cc_meas']
    
    meas_t = list(ca_meas.keys())
       
    ca0 = 1.0
    cb0 = 0.0
    cc0 = 0.0
        
    m = ConcreteModel()
    
    #m.k1 = Var(initialize = 0.5, bounds = (1e-4, 10))
    #m.k2 = Var(initialize = 3.0, bounds = (1e-4, 10))
    m.k1 = Var(initialize = 2.0)
    m.k2 = Var(initialize = 1.0)
    
    m.time = ContinuousSet(bounds = (0.0, 5.0), initialize = meas_t)
    #m.ca = Var(m.time, initialize = ca0, bounds = (0, ca0))
    #m.cb = Var(m.time, initialize = cb0, bounds = (0, ca0))
    #m.cc = Var(m.time, initialize = cc0, bounds = (0, ca0))
    
    m.ca = Var(m.time, initialize = ca0)
    m.cb = Var(m.time, initialize = cb0)
    m.cc = Var(m.time, initialize = cc0)
    
    m.dca = DerivativeVar(m.ca, wrt = m.time)
    m.dcb = DerivativeVar(m.cb, wrt = m.time)
    m.dcc = DerivativeVar(m.cc, wrt = m.time)
    
    def _dcarate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dca[t] == -m.k1 * m.ca[t]
    m.dcarate = Constraint(m.time, rule = _dcarate)
    
    def _dcbrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcb[t] == m.k1 * m.ca[t] - m.k2 * m.cb[t]
    m.dcbrate = Constraint(m.time, rule = _dcbrate)
    
    def _dccrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcc[t] == m.k2 * m.cb[t]
    m.dccrate = Constraint(m.time, rule = _dccrate)
    
    def _initcon(m):
        yield m.ca[m.time.first()] == ca0
        yield m.cb[m.time.first()] == cb0
        yield m.cc[m.time.first()] == cc0
    m.initcon = ConstraintList(rule = _initcon)
    
    def ComputeFirstStageCost_rule(m):
        return 0
    m.FirstStageCost = Expression(rule=ComputeFirstStageCost_rule)

    def ComputeSecondStageCost_rule(m):
        return sum((m.ca[t] - ca_meas[t]) ** 2 + (m.cb[t] - cb_meas[t]) ** 2 
                   + (m.cc[t] - cc_meas[t]) ** 2 for t in meas_t) 
    m.SecondStageCost = Expression(rule=ComputeSecondStageCost_rule)

    
    def total_cost_rule(model):
        return model.FirstStageCost + model.SecondStageCost
    m.Total_Cost_Objective = Objective(rule=total_cost_rule, sense=minimize)
    
    disc = TransformationFactory('dae.collocation')
    disc.apply_to(m, nfe=20, ncp=2)
    
    return m

In [5]:
# parameters to be estimated
theta_names = ['k1', 'k2']

In [6]:
# First tried collocation method. Max iterations are exceeded
pest = parmest.Estimator(ABC_model, data, theta_names, tee = True)
res = pest.theta_est()

Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      927
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      140

Total number of variables............................:      247
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      123
                     variables with only upper bounds:        0
Total number of equality constraints.................:      245
Total number of inequali

In [7]:
m = ABC_model(data[0])

In [8]:
if not hasattr(m, 'ipopt_zL_out'):
    m.ipopt_zL_out = Suffix(direction=Suffix.IMPORT)
if not hasattr(m, 'ipopt_zU_out'):
    m.ipopt_zU_out = Suffix(direction=Suffix.IMPORT)
if not hasattr(m, 'ipopt_zL_in'):
    m.ipopt_zL_in = Suffix(direction=Suffix.EXPORT)
if not hasattr(m, 'ipopt_zU_in'):
    m.ipopt_zU_in = Suffix(direction=Suffix.EXPORT)
if not hasattr(m, 'dual'):
    m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [9]:
solver = SolverFactory('ipopt')
solver.options['bound_relax_factor']=0
solver.options['honor_original_bounds']='no'

In [10]:
status = solver.solve(m, tee=True)

Ipopt 3.12: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      923
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      140

Total number of variables............................:      245
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      123
                     variables with only upper bounds:        0
Total number of equality constraints......

In [11]:
estimated_mu = list()
for v in m.ipopt_zL_out:
    if v.has_lb():
        estimated_mu.append((value(v) - v.lb)*m.ipopt_zL_out[v])
for v in m.ipopt_zU_out:
    if v.has_ub():
        estimated_mu.append((v.ub - value(v))*m.ipopt_zU_out[v])
if len(estimated_mu) == 0:
    mu = 10**-8.6
else:
    mu = sum(estimated_mu)/len(estimated_mu)
    # check to make sure these estimates were all reasonable
    if any([abs(mu-estmu) > 1e-7 for estmu in estimated_mu]):
        print('Warning: estimated values of mu do not seem consistent - using mu=10^(-8.6)')
        mu = 10**-8.6

In [21]:

for i, v in enumerate(m.ipopt_zL_out):
    print(v, v.lb, v.ub, value(v), m.ipopt_zL_out[v], m.ipopt_zU_out[v], estimated_mu[i])

ca[0.263] 0 1.0 0.5881614242907732 1.5455445310164223e-09 -2.2073962061518287e-09 9.090296726674341e-10
ca[0.526] 0 1.0 0.3459338610202313 2.6273422035679286e-09 -1.3899070459650429e-09 9.088866327016562e-10
ca[0.789] 0 1.0 0.2034649524034121 4.466172922812694e-09 -1.141308008507083e-09 9.087096611654927e-10
ca[1.053] 0 1.0 0.1194276840076603 7.607911209993145e-09 -1.0323885760887638e-09 9.085952159453978e-10
ca[1.3159999999999998] 0 1.0 0.07024275672158602 1.2935416375040728e-08 -9.777753193919298e-10 9.08619305524406e-10
ca[1.579] 0 1.0 0.04131407983606198 2.1996749711634225e-08 -9.48271196223346e-10 9.087754737203297e-10
ca[1.8419999999999999] 0 1.0 0.024299348036915896 3.740750197453891e-08 -9.317349592282655e-10 9.089779096709396e-10
ca[2.105] 0 1.0 0.01429193914862044 6.361084235881611e-08 -9.222753392593666e-10 9.091222881846873e-10
ca[2.3680000000000003] 0 1.0 0.00840596728393458 1.0815650742716882e-07 -9.168004442424831e-10 9.091600629774084e-10
ca[2.6319999999999997] 0 1.0 0.

In [18]:
any([abs(mu-estmu) > 1e-7 for estmu in estimated_mu])

False

In [22]:
independent_variables = [m.k1, m.k2]
ind_vardatas = list()
for v in independent_variables:
    if v.is_indexed():
        for k in v:
            ind_vardatas.append(v[k])
    else:
        ind_vardatas.append(v)

In [24]:
for v in ind_vardatas:
    if (v.has_lb() and pyo.value(v) - v.lb <= bound_tolerance) or \
       (v.has_ub() and v.ub - pyo.value(b) <= bound_tolerance):
            raise ValueError("Independent variable: {} has a solution value that is near"
                             " its bound (according to tolerance). The reduced hessian"
                             " computation does not support this at this time. All"
                             " independent variables should be in their interior.".format(v))

In [27]:
kkt_builder = InteriorPointInterface(m)

In [28]:
pyomo_nlp = kkt_builder.pyomo_nlp()

In [29]:
ind_var_indices = pyomo_nlp.get_primal_indices(ind_vardatas)

In [31]:
kkt_builder.set_barrier_parameter(mu)
kkt = kkt_builder.evaluate_primal_dual_kkt_matrix()

/home/paperspace/anaconda3/envs/my_pyomoenv/lib/python3.8/site-packages/pyomo/contrib/interior_point/interface.py:410: RuntimeWarning: divide by zero encountered in true_divide
  duals_primals_ub/(self._nlp.primals_ub() - primals))


In [37]:
primals = kkt_builder._nlp.get_primals()

In [40]:
duals_primals_lb = kkt_builder._duals_primals_lb
duals_primals_ub = kkt_builder._duals_primals_ub

In [46]:
#duals_primals_lb / (primals - kkt_builder._nlp.primals_lb())
duals_primals_ub / (kkt_builder._nlp.primals_ub() - primals)

<ipython-input-46-6206a413d591>:2: RuntimeWarning: divide by zero encountered in true_divide
  duals_primals_ub / (kkt_builder._nlp.primals_ub() - primals)


array([-5.35985781e-09, -2.12502523e-09, -1.43284092e-09, -1.17240635e-09,
       -1.05164582e-09, -9.89136459e-10, -9.54939363e-10, -9.35647557e-10,
       -9.24572369e-10, -9.18131213e-10, -9.14392329e-10, -9.12203759e-10,
       -9.10920079e-10, -9.10166252e-10, -9.09723262e-10, -9.09462390e-10,
       -9.09309418e-10, -9.09219448e-10, -9.09166527e-10, -2.21087555e-09,
       -3.45689589e-09, -3.62410401e-09, -3.08379666e-09, -2.48118091e-09,
       -2.01939644e-09, -1.69721772e-09, -1.47574051e-09, -1.32197456e-09,
       -1.21311357e-09, -1.13536368e-09, -1.07877604e-09, -1.03709622e-09,
       -1.00610001e-09, -9.82872213e-10, -9.65305847e-10, -9.52055679e-10,
       -9.41982617e-10, -9.34303112e-10,  0.00000000e+00,  0.00000000e+00,
       -3.57095567e-08, -3.55509174e-09, -1.80715099e-09, -1.32246697e-09,
       -1.12326645e-09, -1.02674347e-09, -9.75673064e-10, -9.47395147e-10,
       -9.31333331e-10, -9.22067719e-10, -9.16679996e-10, -9.13543513e-10,
       -9.11706124e-10, -

In [48]:
duals_primals_ub[82], kkt_builder._nlp.primals_ub()[82], primals[82]

(-3629.5935101309587, 1.0, 1.0)